In [25]:
# import necessary library
import pandas as pd
import numpy as np
import os
import glob
from fuzzywuzzy import process

In [26]:
# read the raw data that needs to be process
rawd=pd.read_excel('Z:/AT Data/Tableau Database/Utility/For Mintu/D1/Copy of NM Count Summary Master.xlsx')
rawd.head()

,County,Route,PM,LocName,City,C_R_PM,C_R_PM+Loc,Date,Day,Month,...,PedsSB,PedsEB,PedsWB,BikAll_Dir,BikesNB,BikesSB,BikesEB,BikesWB,Video Stored by Miovision? (Y/N),Deleted Caltrans Video? (Y/N)
0,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2014-07-31,Thu,7,...,47.0,70.0,66.0,68,4.0,28.0,30.0,6.0,N,N
1,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2014-08-02,Sat,8,...,48.0,58.0,68.0,43,0.0,17.0,19.0,7.0,N,N
2,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2014-08-03,Sun,8,...,28.0,56.0,46.0,34,3.0,14.0,9.0,8.0,N,N
3,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2019-08-15,Thu,8,...,29.0,55.0,48.0,44,0.0,15.0,16.0,13.0,Y,N
4,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2019-08-17,Sat,8,...,19.0,32.0,20.0,25,1.0,14.0,6.0,4.0,Y,N


In [27]:
rawd.columns

Index(['County', 'Route', 'PM', 'LocName', 'City', 'C_R_PM', 'C_R_PM+Loc',
       'Date', 'Day', 'Month', 'Year', 'TOD', 'By', 'Type', 'Modes', 'Reason',
       'Data Status', 'GIS_ID2', 'GIS_ID', 'Comm', 'PedAll_Dir', 'PedsNB',
       'PedsSB', 'PedsEB', 'PedsWB', 'BikAll_Dir', 'BikesNB', 'BikesSB',
       'BikesEB', 'BikesWB', 'Video Stored by Miovision? (Y/N)',
       'Deleted Caltrans Video? (Y/N)'],
      dtype='object')

In [28]:
# keep only intersection data
rawd1=rawd.loc[rawd['Type']=='TMC']
# add count type and district
rawd1['Count_Type']= 'Short term'
rawd1['District']= 1
# assign the unique ID for each location
# Create a dictionary to map each unique location to a unique ID starting from 1001
unique_locations = rawd1['LocName'].unique()
location_to_id = {LocName: 1001 + idx for idx, LocName in enumerate(unique_locations)}
# Map the 'Location' column to the new ID
rawd1['Loc_id'] = rawd1['LocName'].map(location_to_id)
rawd1.head()

C:\Users\s160555\AppData\Local\Temp\ipykernel_18320\3929752054.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawd1['Count_Type']= 'Short term'
C:\Users\s160555\AppData\Local\Temp\ipykernel_18320\3929752054.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawd1['District']= 1
C:\Users\s160555\AppData\Local\Temp\ipykernel_18320\3929752054.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,County,Route,PM,LocName,City,C_R_PM,C_R_PM+Loc,Date,Day,Month,...,BikAll_Dir,BikesNB,BikesSB,BikesEB,BikesWB,Video Stored by Miovision? (Y/N),Deleted Caltrans Video? (Y/N),Count_Type,District,Loc_id
0,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2014-07-31,Thu,7,...,68,4.0,28.0,30.0,6.0,N,N,Short term,1,1001
1,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2014-08-02,Sat,8,...,43,0.0,17.0,19.0,7.0,N,N,Short term,1,1001
2,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2014-08-03,Sun,8,...,34,3.0,14.0,9.0,8.0,N,N,Short term,1,1001
3,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2019-08-15,Thu,8,...,44,0.0,15.0,16.0,13.0,Y,N,Short term,1,1001
4,HUM,101,79.44,"4th and V St, Eureka",Eureka,HUM-101-79.44,"HUM-101-79.44, 4th and V St, Eureka",2019-08-17,Sat,8,...,25,1.0,14.0,6.0,4.0,Y,N,Short term,1,1001


In [57]:
# drop the unnecessary data
rawd2=rawd1.drop(['County', 'Route', 'PM','City', 'C_R_PM', 'C_R_PM+Loc','Reason','Video Stored by Miovision? (Y/N)',
       'Deleted Caltrans Video? (Y/N)','By','Data Status', 'GIS_ID2', 'GIS_ID', 'Comm'],axis=1)
# rename some of the columns that match with tableau format
rawd2=rawd2.rename(columns={'LocName': "Intersection",'TOD':'Hour'})
rawd2.head()

,Intersection,Date,Day,Month,Year,Hour,Type,Modes,PedAll_Dir,PedsNB,...,PedsEB,PedsWB,BikAll_Dir,BikesNB,BikesSB,BikesEB,BikesWB,Count_Type,District,Loc_id
0,"4th and V St, Eureka",2014-07-31,Thu,7,2014,7AM - 7PM,TMC,Bike Ped Auto,223,40.0,...,70.0,66.0,68,4.0,28.0,30.0,6.0,Short term,1,1001
1,"4th and V St, Eureka",2014-08-02,Sat,8,2014,7AM - 7PM,TMC,Bike Ped Auto,228,54.0,...,58.0,68.0,43,0.0,17.0,19.0,7.0,Short term,1,1001
2,"4th and V St, Eureka",2014-08-03,Sun,8,2014,7AM - 7PM,TMC,Bike Ped Auto,169,39.0,...,56.0,46.0,34,3.0,14.0,9.0,8.0,Short term,1,1001
3,"4th and V St, Eureka",2019-08-15,Thu,8,2019,7AM - 7PM,TMC,Bike Ped Auto,159,27.0,...,55.0,48.0,44,0.0,15.0,16.0,13.0,Short term,1,1001
4,"4th and V St, Eureka",2019-08-17,Sat,8,2019,7AM - 7PM,TMC,Bike Ped Auto,105,34.0,...,32.0,20.0,25,1.0,14.0,6.0,4.0,Short term,1,1001


In [58]:
rawd2.Modes.unique()

array(['Bike Ped Auto', 'Bike Ped'], dtype=object)

# Lets separate the bike 

In [77]:
# select the bike data only
bdata=rawd2[['Intersection', 'Date', 'Month', 'Year', 'Hour', 'BikAll_Dir',
       'BikesNB', 'BikesSB', 'BikesEB', 'BikesWB', 'District',
       'Loc_id','Count_Type']]
#rename the bound columns
bdata1=bdata.rename(columns={'BikesNB': "Northbound",'BikesEB':'Eastbound','BikesWB':'Westbound','BikesSB':'Southbound'})
# assign the mode
bdata1['Mode']='Bicycle'
# melt the dataframe
bdata_melted = pd.melt(
    bdata1, 
    id_vars=['Intersection', 'Date', 'Month', 'Year', 'Hour', 'District', 'Loc_id','Mode','Count_Type'], 
    value_vars=['Northbound', 'Eastbound', 'Westbound', 'Southbound'], 
    var_name='Direction', 
    value_name='Count'
)
# make sure date is in date formate
bdata_melted['Date'] = pd.to_datetime(bdata_melted['Date'], format="%Y/%m/%d")
bdata_melted['Day'] = bdata_melted['Date'].dt.day_name()
#assign weekday type
bdata_melted['Weekday_Type'] = bdata_melted['Day'].apply(lambda x: "Weekend" if x in ['Saturday', 'Sunday'] else "Weekday")
# re-order the columns
bdata_melted1=bdata_melted[['Loc_id','District','Intersection','Year', 'Month',
       'Day','Weekday_Type','Date', 'Hour','Mode','Direction','Count','Count_Type']]
# check for the dupicates
bdata_melted2=bdata_melted1.drop_duplicates(subset=['Loc_id','District','Intersection','Year', 'Month',
       'Day','Weekday_Type','Date', 'Hour','Mode','Direction','Count','Count_Type'])
bdata_melted2=bdata_melted2.dropna(subset=['Count'])
bdata_melted2.head()


,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Hour,Mode,Direction,Count,Count_Type
0,1001,1,"4th and V St, Eureka",2014,7,Thursday,Weekday,2014-07-31,7AM - 7PM,Bicycle,Northbound,4.0,Short term
1,1001,1,"4th and V St, Eureka",2014,8,Saturday,Weekend,2014-08-02,7AM - 7PM,Bicycle,Northbound,0.0,Short term
2,1001,1,"4th and V St, Eureka",2014,8,Sunday,Weekend,2014-08-03,7AM - 7PM,Bicycle,Northbound,3.0,Short term
3,1001,1,"4th and V St, Eureka",2019,8,Thursday,Weekday,2019-08-15,7AM - 7PM,Bicycle,Northbound,0.0,Short term
4,1001,1,"4th and V St, Eureka",2019,8,Saturday,Weekend,2019-08-17,7AM - 7PM,Bicycle,Northbound,1.0,Short term


In [78]:
bdata_melted2.shape

(581, 13)

In [79]:
bdata_melted2['Count'].isnull().sum()

0

# now lets process the ped data

In [80]:
# select the bike data only
pdata=rawd2[['Intersection', 'Date', 'Month', 'Year', 'Hour', 'PedAll_Dir', 'PedsNB',
       'PedsSB', 'PedsEB', 'PedsWB', 'Count_Type', 'District',
       'Loc_id']]
#rename the bound columns
pdata1=pdata.rename(columns={'PedsNB': "Northbound",'PedsEB':'Eastbound','PedsWB':'Westbound','PedsSB':'Southbound'})
# assign the mode
pdata1['Mode']='Pedestrian'
# melt the dataframe
pdata_melted = pd.melt(
    pdata1, 
    id_vars=['Intersection', 'Date', 'Month', 'Year', 'Hour','Count_Type', 'District', 'Loc_id','Mode'], 
    value_vars=['Northbound', 'Eastbound', 'Westbound', 'Southbound'], 
    var_name='Direction', 
    value_name='Count'
)
# make sure date is in date formate
pdata_melted['Date'] = pd.to_datetime(pdata_melted['Date'], format="%Y/%m/%d")
pdata_melted['Day'] = pdata_melted['Date'].dt.day_name()
#assign weekday type
pdata_melted['Weekday_Type'] = pdata_melted['Day'].apply(lambda x: "Weekend" if x in ['Saturday', 'Sunday'] else "Weekday")
# re-order the columns
pdata_melted1=pdata_melted[['Loc_id','District','Intersection','Year', 'Month',
       'Day','Weekday_Type','Date', 'Hour','Mode','Direction','Count','Count_Type',]]
# check for the dupicates
pdata_melted2=pdata_melted1.drop_duplicates(subset=['Loc_id','District','Intersection','Year', 'Month',
       'Day','Weekday_Type','Date', 'Hour','Mode','Direction','Count','Count_Type',])
pdata_melted2=pdata_melted2.dropna(subset=['Count'])
pdata_melted2.head()

,Loc_id,District,Intersection,Year,Month,Day,Weekday_Type,Date,Hour,Mode,Direction,Count,Count_Type
0,1001,1,"4th and V St, Eureka",2014,7,Thursday,Weekday,2014-07-31,7AM - 7PM,Pedestrian,Northbound,40.0,Short term
1,1001,1,"4th and V St, Eureka",2014,8,Saturday,Weekend,2014-08-02,7AM - 7PM,Pedestrian,Northbound,54.0,Short term
2,1001,1,"4th and V St, Eureka",2014,8,Sunday,Weekend,2014-08-03,7AM - 7PM,Pedestrian,Northbound,39.0,Short term
3,1001,1,"4th and V St, Eureka",2019,8,Thursday,Weekday,2019-08-15,7AM - 7PM,Pedestrian,Northbound,27.0,Short term
4,1001,1,"4th and V St, Eureka",2019,8,Saturday,Weekend,2019-08-17,7AM - 7PM,Pedestrian,Northbound,34.0,Short term


In [81]:
pdata_melted2.shape

(567, 13)

In [82]:
pdata_melted2.isnull().sum()

Loc_id          0
District        0
Intersection    0
Year            0
Month           0
Day             0
Weekday_Type    0
Date            0
Hour            0
Mode            0
Direction       0
Count           0
Count_Type      0
dtype: int64

# Now combine bike and ped data togetehr


In [83]:
d1sdata=pd.concat([pdata_melted2,bdata_melted2],axis=0)
d1sdata.shape

(1148, 13)

In [84]:
d1sdata.to_csv('Z:/AT Data/Tableau Database/Utility/For Mintu/D1/D1_bike_ped_short_term_combined_tableau_data.csv',index=False)

In [85]:
# end of the script